# Electromagnetic field evolution in 1D

This notebook introduces the fundamental principles of how the evolution of the electric and magnetic fields over time can be computed. As a first case, we will ignore electric charges (charged particles) and thus currents and stick to one-dimensional simulations. Later notebooks will extend on the algorithms presented here. 

### Remark:
All notebooks here use [`python`](https://www.python.org/) as a programming language. This is not the most performant choice since `python` is an interpreted language. To make matters even worse, we will not use efficient functions (written in C and Fortran) form modules like [`numpy`](https://www.numpy.org/) in order to demonstrate the algorithms clearly and not obfuscate them by writing them as efficient `numpy` function calls. 

## Maxwell's equations

Classical electrodynamics, the interplay between electric and magnetic fields and how they are changed based on charges and currents, is described by the four [Maxwell's equations](https://en.wikipedia.org/wiki/Maxwell%27s_equations).

$
\begin{eqnarray}
\vec{\nabla} \vec{E} & = & \frac{\rho}{\epsilon_0} \tag{1}\\
\vec{\nabla} \vec{B} & = & 0 \tag{2} \\
\vec{\nabla} \times \vec{E} & = & - \frac{\partial \vec{B}}{\partial t} \tag{3} \\
\vec{\nabla} \times \vec{B} & = & \mu_0 \vec{J} - \mu_0 \epsilon_0 \frac{\partial \vec{E}}{\partial t} \tag{4}
\end{eqnarray}
$

with $\vec{E}$ being the electric and $\vec{B}$ being the magnetic field. $\vec{J}$ is the current and $\rho$ the charge density. The differential operator $\vec{\nabla}$ defines the divergence and rotation and can be written as 

$$
\vec{\nabla} = \begin{bmatrix}\vec{e}_x \frac{\partial}{\partial x} \\  \vec{e}_y \frac{\partial}{\partial y} \\ \vec{e}_z \frac{\partial}{\partial z} \end{bmatrix} 
$$

$\epsilon_0$ and $\mu_0$ are constants. The first is called [vacuum permittivity](https://en.wikipedia.org/wiki/Vacuum_permittivity) or electric constant. The second is named the [vacuum permeability](https://en.wikipedia.org/wiki/Vacuum_permeability) or magnetic constant. They are related to the speed of light $c$ by the following relationship:

$$
c^2 = \frac{1}{\epsilon_0 \mu_0}
$$


### Time evolution

Assuming we have a valid solution of Maxwell's equation for both the electric $\vec{E}(\vec{r},t)$ and magnetic field $\vec{B}(\vec{r},t)$ for the entire  volume of interest for time $t = 0$. 
Based on that description, we need to integrate the time evolution of the electric and magnetic fields to predict their evolution over time. 
The 3rd and 4th Maxwell's equations contain a time derivate of the fields.  If we rearrange both equation, we get the following system of coupled particle differential equations ([PDE](https://en.wikipedia.org/wiki/Partial_differential_equation)):

$
\begin{eqnarray}
\frac{\partial \vec{E}}{\partial t} & = & - \frac{1}{c^2} \vec{\nabla} \times \vec{B} + \frac{1}{\epsilon_0} \vec{J} \tag{5}\\
\frac{\partial \vec{B}}{\partial t} & = & - \vec{\nabla} \times \vec{E} \tag{6}\\
\end{eqnarray}
$

By integrating the time evolution, we can determine the field distribution at a later (earlier) time. 

$$
\begin{eqnarray}
\vec{E}(t) & = & \vec{E}(0) + \int \limits_0^{t} \left( - \frac{1}{c^2} \vec{\nabla} \times \vec{B}(\tau) + \frac{1}{\epsilon_0} \vec{J}(\tau) \right) \mathrm{d}\tau \\
\vec{B}(t) & = & \vec{B}(0) + \int \limits_0^{t} \left( - \vec{\nabla} \times \vec{E} \right) \mathrm{d}\tau \\
\end{eqnarray}
$$

### Component-wise formulation of the  Maxwell equation

To keep things simple, we will focus on a one-dimensional problem first - an electromagnetic wave propagating in the $\vec{e}_z$ direction. For that let us write down Maxwell's equation in a component-wise formulation. 

$$
\frac{\partial \vec{E}}{\partial t} = \begin{pmatrix} \frac{\partial E_x}{\partial t} \\ \frac{\partial E_y}{\partial t} \\ \frac{\partial E_z}{\partial t}\end{pmatrix} = - \frac{1}{c^2} \left| \begin{pmatrix} \vec{e}_x && \vec{e}_y && \vec{e}_z \\ \frac{\partial}{\partial x} && \frac{\partial}{\partial y} && \frac{\partial}{\partial z} \\ B_x && B_y && B_z \end{pmatrix}  \right| + \frac{1}{\varepsilon_0} \begin{pmatrix} J_x \\ J_y \\ J_z \end{pmatrix}
$$

$$
\frac{\partial \vec{B}}{\partial t} = \begin{pmatrix} \frac{\partial B_x}{\partial t} \\ \frac{\partial B_y}{\partial t} \\ \frac{\partial B_z}{\partial t}\end{pmatrix} = - \left| \begin{pmatrix} \vec{e}_x && \vec{e}_y && \vec{e}_z \\ \frac{\partial}{\partial x} && \frac{\partial}{\partial y} && \frac{\partial}{\partial z} \\ E_x && E_y && E_z \end{pmatrix}  \right|
$$

With $|\dots|$ denoting the [determinate](https://en.wikipedia.org/wiki/Determinant) as represenation of the [curl](https://en.wikipedia.org/wiki/Curl_(mathematics)#Usage).

In this one-dimensional case, there is no change of any quantity directions other than $\vec{e}_z$. Thus, both the spatial derivative $\frac{\partial}{\partial x}$ and $\frac{\partial}{\partial y}$ will return zero. Maxell's equation thus simplifies to:

$$
 \begin{pmatrix} \frac{\partial E_x}{\partial t} \\ \frac{\partial E_y}{\partial t} \\ \frac{\partial E_z}{\partial t}\end{pmatrix} = - \frac{1}{c^2}  \begin{pmatrix} - \frac{\partial B_y}{\partial z} \\ \frac{\partial B_x}{\partial z} \\ 0 \end{pmatrix}  + \frac{1}{\varepsilon_0} \begin{pmatrix} J_x \\ J_y \\ J_z \end{pmatrix}
$$

$$
 \begin{pmatrix} \frac{\partial B_x}{\partial t} \\ \frac{\partial B_y}{\partial t} \\ \frac{\partial B_z}{\partial t}\end{pmatrix} = -  \begin{pmatrix} - \frac{\partial E_y}{\partial z} \\ \frac{\partial E_x}{\partial z} \\ 0  \end{pmatrix} 
$$

The $B_z$ componet will not change over time. 
$$
B_z(\tau) = B_z(0) + \int \limits_0^{\tau} \operatorname{d}t \frac{\partial B_z}{\partial t} =  B_z(0)
$$

The $E_z$ component will only change over time, if there is a current $J_z$. 

### Assuming vacuum - no electric current

To further simplify our first implementation, let us assume that there are no charged particles and therefore no current. This lead us to the following set of equation, with $\dot{a}$ representiang the time derivative $\frac{\partial a}{\partial t}$.

$$
\begin{eqnarray}
\dot E_x & = & + \frac{1}{c^2} \frac{\partial B_y}{\partial z} \\
\dot B_y & = & - \frac{\partial E_x}{\partial z} \\
& &\\
\dot E_y & = & - \frac{1}{c^2} \frac{\partial B_x}{\partial z} \\
\dot B_x & = & + \frac{\partial E_y}{\partial z} \\
\end{eqnarray}
$$

Please note that the first two and the last two equations are coupled with each other. In a general case, with electric current, an electric field $E_x$ will cause a current $J_x$ which in turn changes $E_y$, thus coupling the equations with each other. But for our simple vacuum case, we can separate both equation pairs and handle them independently. 

## Numerically solving Maxwell's equation in 1D

In the previous section, we derive a set of 4 coupled differential equations, that we now need to numerically integrate to get a solution of the electric and magnetic fields over time. The left-hand side gives us a time derivative of each field, which we will integrate in steps of $\Delta t$. The right-hand side contains spatial derivatives. We will explore various numerical differentiation equation to finally arrive at the so-called staggered grid introduced by [Yee in 1966](https://doi.org/10.1109/TAP.1966.1138693) that is the most common method in particle-in-cell codes today. 

For simplicity, we will assume that $E_y$ and $B_x$ are constant in time and space, thus we only need to focus on the $E_x$ and $B_y$ set of equations.

### Periodic boundary conditions

In the following sections, we will discretize the electric field $E_x$ and the magnetic field $B_z$ on a grid. Depending on the method we choose, the grid spacing will be defined differently. But for our initial simple setup, let us assume, that the left-most grid point has a left-side neighbor that is defined by the right-most grid point and vice versa. This we assume that the finite sample of electric and magnetic fields that we discretized reoccur periodically. This is not a necessary requirement but will make thinks simpler initially. 

### The Courant–Friedrichs–Lewy condition
Without going into too much detail, the [Courant-Friedrich-Lewy condition](https://en.wikipedia.org/wiki/Courant%E2%80%93Friedrichs%E2%80%93Lewy_condition) defines a criterion on how large a time step can be when solving particle differential equations. We will explore the numerical consequences of not fulfilling this condition later, but for now, let us briefly have a look at the condition itself. 

$$
C = v \cdot \frac{\Delta t}{\Delta z} \leq C_\mathrm{max}
$$

Where $C$ is the Corant number, which needs to be below a critical threshold $C_\mathrm{max}$ - which is usually $1$ for explicit methods as we will use in the following. 
$\Delta t$ defines the stepsize for (time) integration, while $\Delta z$ defines the (spatial) distance for numerical differentiation. $v$ is the speed at which (in our case) the electric and magnetic field can propagate - thus the speed of light $c$.  

### Forward (one-sided) differentiation

The forward and backward difference is the simplest form of [fimite differences](https://en.wikipedia.org/wiki/Finite_difference) to perform [numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation). Here we will focus on the **forward differentiation**:

$$
\frac{\partial f(z)}{\partial z} \approx \frac{f(z+\Delta z) - f(z)}{\Delta z}
$$


Thus with $c=1$, the pair of Maxwell equation allows to determine the electric and magnetic field for a future time $t + \Delta t$:
$$
\begin{eqnarray}
E_x (t + \Delta t, z) & = & \frac{B_y(t, z+ \Delta z) - B_y(t, z)}{\Delta z} \cdot \Delta t + E_x(t, z)\\
B_y (t + \Delta t, z) & = &  - \frac{E_x (t, z + \Delta z) - E_x(t, z)}{\Delta z} \cdot \Delta t + B_y(t, z)
\end{eqnarray}
$$

We discretize the positions $z$ positions as
$$
z_n = \Delta z \cdot n \quad n \in \mathbb{N} , 0\leq n < N_z
$$
and the time $t$ as
$$
t_k = \Delta t \cdot k \quad k \in \mathbb{N} , 0\leq k < N_t
$$

With that we can define
$$
E_x(t_k, z_n) = E_x^{(k,n)} \quad \mathrm{and} \quad B_y(t_k, z_n) = B_y^{(k,n)}
$$
thus alowing the rewrite the numerical version of Maxwell's equation as:
$$
\begin{eqnarray}
E_x^{(k+1,n)} & = & \frac{\Delta t}{\Delta z} \cdot \left( B_y^{(k,n+1)} - B_y^{(k,n)} \right) + E_x^{(k, n)}\\
B_y^{(k+1,n)} & = & -\frac{\Delta t}{\Delta z} \cdot \left( E_x^{(k,n+1)} - E_x^{(k,n)} \right) + B_y^{(k, n)}\\
\end{eqnarray}
$$

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
N_t = 60*10
N_z = 1000

delta_t = 0.01
delta_z = 0.1
ratio = delta_t/delta_z

z = np.arange(N_z) * delta_z
N_waves=3
k_0 = 2 * np.pi / (delta_z * N_z) * N_waves
Ex_0 = np.sin(k_0 * z)
By_0 = np.sin(k_0 * z)




Ex_all = np.zeros((N_t, N_z))
By_all = np.zeros((N_t, N_z))

Ex_all[0, :] = Ex_0
By_all[0, :] = By_0

for k in range(0, N_t-1):
    for n in range(N_z):
        n_plus_1 = (n+1) % N_z
        n_minus_1 = (n-1) % N_z
        
        Ex_all[k+1, n] = +ratio * 0.5 * (By_all[k, n_plus_1] - By_all[k, n_minus_1]) + Ex_all[k, n]
        By_all[k+1, n] = -ratio * 0.5 * (Ex_all[k, n_plus_1] - Ex_all[k, n_minus_1]) + By_all[k, n]
        
        

In [ ]:
plt.imshow(Ex_all, vmin=-1, vmax=1, aspect="auto", cmap=plt.cm.RdBu, origin="lower" )
plt.colorbar()
plt.plot(np.arange(100), np.arange(100))

In [ ]:
plt.plot(Ex_all[0, :])
plt.plot(Ex_all[42, :])

In [ ]:
plt.plot(By_all[0, :])
plt.plot(By_all[30, :])

In [ ]:
plt.plot(By_all[0, 1:] - By_all[0, :-1])